In [1]:
!pip install opencv-python

In [9]:
import numpy as np
import os
import cv2
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.optimizers import Adam, Adadelta
#from keras.utils import  Adam, Adadelta
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


In [10]:
image_size = 50
image_number = 1000
folder = "../data/SolarPanelSoilingImageDataset/Solar_Panel_Soiling_Image_dataset/PanelImages"
image_width = 192
image_height = 192
ratio = 1

image_width = int(image_width / ratio)
image_height = int(image_height / ratio)

channels = 3

In [11]:
def read_and_normalize_train_data(image_number):
    onlyfiles = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    onlyfiles = onlyfiles[0:image_number]
    train_data, train_target = load_train(onlyfiles)
    train_data = np.array(train_data, dtype=np.float32)
    #train_target = np.array(train_target, dtype=np.float32)
    #m = train_data.mean()
    #s = train_data.std()

    #print ('Train mean, sd:', m, s )
    #train_data -= m
    #train_data /= s
    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target

In [12]:
X_train = np.ndarray(shape=(1, image_height, image_width, channels),
                     dtype=np.float32)

In [13]:
X_train.shape

(1, 192, 192, 3)

In [14]:
_file = onlyfiles[0]

NameError: name 'onlyfiles' is not defined

In [15]:
img = load_img(folder + "/" + _file)  # this is a PIL image

NameError: name '_file' is not defined

In [16]:
x = img_to_array(img)

NameError: name 'img' is not defined

In [17]:
type(x)

NameError: name 'x' is not defined

In [18]:
def load_train(onlyfiles):
    print('Read train images')
    X_train = np.ndarray(shape=(len(onlyfiles), image_height, image_width, channels),
                     dtype=np.float32)
    y_train = np.ndarray(len(onlyfiles))
    i=0
    for _file in onlyfiles:
        y_train[i] = (np.float64(_file.split('_')[11]))     
        img = load_img(folder + "/" + _file)  # this is a PIL image
        x = img_to_array(img)  
        X_train[i] = x
        i += 1
        if i % 250 == 0:
            print("%d images to array" % i)
    return X_train, y_train

In [19]:
def create_model():
    nb_filters = 8
    nb_conv = 5

    model = Sequential()
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                            input_shape=(3, image_size, image_size) ) )
    model.add(Activation('relu'))

    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))

    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))

    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv))
    model.add(Activation('relu'))

    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv))
    model.add(Activation('relu'))

    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv))
    model.add(Activation('relu'))

    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('linear'))

    model.compile(loss='mean_squared_error', optimizer=Adadelta())
    return model



In [20]:
def train_model(batch_size = 50, nb_epoch = 20, images_nr = 1000):
    num_samples = 1999
    cv_size = 499

    train_data, train_target = read_and_normalize_train_data(images_nr)
    train_data = train_data[0:num_samples,:,:,:]
    train_target = train_target[0:num_samples]

    X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=cv_size, random_state=56741)

    model = create_model()
    model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, validation_data=(X_valid, y_valid) )

    predictions_valid = model.predict(X_valid, batch_size=50, verbose=1)
    compare = pd.DataFrame(data={'original':y_valid.reshape((cv_size,)),
             'prediction':predictions_valid.reshape((cv_size,))})
    compare.to_csv('compare.csv')

    return model

In [21]:
train_model(nb_epoch = 50)

Read train images
250 images to array
500 images to array
750 images to array
1000 images to array
Train shape: (1000, 192, 192, 3)
1000 train samples


ValueError: Negative dimension size caused by subtracting 5 from 3 for '{{node conv2d/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 5, 5, 1], use_cudnn_on_gpu=true](Placeholder, conv2d/Conv2D/ReadVariableOp)' with input shapes: [?,3,50,50], [5,5,50,8].